In [1]:
import streamlit as st
import pandas as pd
import requests
import pickle

# Use st.cache_data to cache the loaded data, so it's only loaded once.
@st.cache_data
def load_data():
    try:
        with open('movie_data.pkl', 'rb') as file:
            movies, cosine_sim = pickle.load(file)
        return movies, cosine_sim
    except FileNotFoundError:
        st.error("Error: 'movie_data.pkl' file not found. Please ensure the data file is in the same directory.")
        return None, None

# Function to get movie recommendations
def get_recommendations(title, movies_df, cosine_sim):
    try:
        idx = movies_df[movies_df['title'] == title].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Get top 10 similar movies
        movie_indices = [i[0] for i in sim_scores]
        return movies_df.iloc[movie_indices]
    except IndexError:
        st.error("Movie not found in the dataset. Please select a different movie.")
        return pd.DataFrame() # Return an empty DataFrame on error

# Fetch movie poster from TMDB API
def fetch_poster(movie_id):
    api_key = '7b995d3c6fd91a2284b4ad8cb390c7b8'  # Replace with your TMDB API key
    if not api_key:
        st.warning("TMDB API key not set. Posters cannot be displayed.")
        return "https://placehold.co/500x750/cccccc/333333?text=No+Poster"
    
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()
        poster_path = data.get('poster_path')
        if poster_path:
            return f"https://image.tmdb.org/t/p/w500{poster_path}"
        else:
            return "https://placehold.co/500x750/cccccc/333333?text=No+Poster"
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching poster: {e}")
        return "https://placehold.co/500x750/cccccc/333333?text=No+Poster"


# --- Streamlit UI Layout ---
st.set_page_config(
    page_title="Movie Recommendation System",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("🎬 Movie Recommendation System")

# Load data only once using the cached function
movies, cosine_sim = load_data()

# Check if data was loaded successfully
if movies is not None and not movies.empty:
    selected_movie = st.selectbox("Select a movie:", movies['title'].values, index=0)

    if st.button('Recommend'):
        with st.spinner('Fetching recommendations...'):
            recommendations = get_recommendations(selected_movie, movies, cosine_sim)

        if not recommendations.empty:
            st.markdown("---")
            st.subheader("Top 10 Recommended Movies:")

            # Create a flexible grid layout for posters
            cols = st.columns(5) # Create 5 columns per row
            
            for i in range(10):
                if i < len(recommendations):
                    movie_title = recommendations.iloc[i]['title']
                    movie_id = recommendations.iloc[i]['movie_id']
                    
                    # Get the poster URL and display it
                    poster_url = fetch_poster(movie_id)
                    
                    with cols[i % 5]: # Use modulo to cycle through columns
                        st.image(poster_url, caption=movie_title, use_column_width=True)
                        st.markdown(f"**{movie_title}**", unsafe_allow_html=True)
                        st.markdown("---")
        else:
            st.info("No recommendations found for this movie.")

2025-09-20 10:05:04.802 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-20 10:05:10.880 
  command:

    streamlit run C:\Users\saran\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-20 10:05:10.895 No runtime found, using MemoryCacheStorageManager
